# Linear Algebra

That linear algebra is fun, is a widely accepted fact. This notebooks will guide you through some of the linear algebra fun you can realize with Heat. 

In [ ]:
from ipyparallel import Client
rc = Client(profile="default")
rc.ids

if len(rc.ids) == 0:
    print("No engines found")
else:
    print(f"{len(rc.ids)} engines found")

In [ ]:
%%px
import heat as ht

## Matrix-Matrix Multiplication

The most basic operation in linear algebra is matrix-matrix multiplication ("matmul"). Doing it by hand for a small matrix is not difficult and in fact not very spectacular. However, in the distributed setting (e.g., on 4 GPUs) even such a simple operation is not trivial any more: just imagine you work together with 3 other people and each of you only knows one fourth of the columns of a matrix $A$ and one fourth of the rows of a matrix $B$. Together, you have to compute the product $AB$ such that in the end each of you only has one fourth of the columns of $AB$...

In [ ]:
%%px
split_A=0 
split_B=1 
M = 10000
N = 10000
K = 10000
A = ht.random.randn(M, N, split=split_A, device="gpu")
B = ht.random.randn(N, K, split=split_B, device="gpu")
C = ht.matmul(A, B)
C

## QR Decomposition and Triangular Solve

Given a matrix $A$, its QR decomposition is given by $A=QR$ where $Q$ is an orthogonal matrix (i.e. its columns are pairwise orthonormal) and $R$ is an upper triangular matrix. 

Further information: [QR on Wikipedia](https://en.wikipedia.org/wiki/QR_decomposition)

In [ ]:
%%px
A = ht.random.randn(100000, 1000, split=0, device="gpu")
Q,R = ht.linalg.qr(A)

With a little bit of linear algebra fun, you find out that a linear least squares problem of type $\min \lVert Ax - b \rVert_2$ boils down to computing the QR decomposition $A=QR$ and then solving for $Rx = Q^T b$. (Of course, we need to assume that if $A \in \mathbb{R}^{m \times n}$ that $m \geq n$ and $R$ is invertible...)

In [ ]:
%%px
b = ht.random.randn(100000,split=None, device="gpu")
Qtb = Q.T @ b
x = ht.linalg.solve_triangular(R,Qtb) 

If you want to solve a LASSO-regularized version of this linear regression problem, try out `heat.regression.Lasso`!

## Singular Value Decomposition

Let $X \in \mathbb{R}^{m \times n}$ be a matrix, e.g., given by a data set consisting of $m$ data points $\in \mathbb{R}^n$ stacked together. The so-called **singular value decomposition (SVD)** of $X$ is given by 

$$
	X = U \Sigma V^T
$$

where $U \in \mathbb{R}^{m \times r_X}$ and $V \in \mathbb{R}^{n \times r_X}$ have orthonormal columns, $\Sigma = \text{diag}(\sigma_1,...,\sigma_{r_X}) \in \mathbb{R}^{r_X \times r_X}$ is a diagonal matrix containing the so-called singular values $\sigma_1 \geq \sigma_2 \geq ... \geq \sigma_{r_X} > 0$, and $r_X \leq \min(m,n)$ denotes the rank of $X$ (i.e. the dimension of the subspace of $\mathbb{R}^m$ spanned by the columns of $X$). Since $\Sigma = U^T X V$ is diagonal, one can imagine this decomposition as finding orthogonal coordinate transformations under which $X$ looks "linear". Further information: [SVD on Wikipedia](https://en.wikipedia.org/wiki/Singular_value_decomposition)

Computing the **full** SVD in a distributed environment can be quite **expensive**; nevertheless, we have implemented it: 

In [ ]:
%%px
A = ht.random.rand(2000,2000, split=1, device="gpu")
U, S, V = ht.linalg.svd(A)
U, S, V

If the number of rows is much higher than the number of columns (we call such matrices "tall-skinny"), a more efficient implementation of SVD is available than in the general case:

In [ ]:
%%px 
X = ht.random.rand(100000,2000, split=0, device="gpu")
U, S, V = ht.linalg.svd(X) 
U, S, V

## Truncated SVD 

So, what to do if your matrix is very large but not tall-skinny? 

Let us start with a brief background on how SVD is actually used in applications... In data science, SVD is more often known as **principle component analysis (PCA)**, the columns of $U$ being called the principle components of $X$. In fact, in many applications **truncated SVD/PCA** suffices: to reduce $X$ to the "essential" information, one chooses a truncation rank $0 < r \leq r_X$ and considers the truncated SVD/PCA given by 

$$
X \approx X_r := U_{[:,:r]} \Sigma_{[:r,:r]} V_{[:,:r]}^T
$$

where we have used `numpy`-like notation for selecting only the first $r$ columns of $U$ and $V$, respectively. The rationale behind this is that if the first $r$ singular values of $X$ are much larger than the remaining ones, $X_r$ will still contain all "essential" information contained in $X$; in mathematical terms: 

$$
\lVert X_r - X \rVert_{F}^2 = \sum_{i=r+1}^{r_X} \sigma_i^2, 
$$

where $\lVert \cdot \rVert_F$ denotes the Frobenius norm. Thus, truncated SVD/PCA may be used for, e.g.,  
* filtering away non-essential information in order to get a "feeling" for the main characteristics of your data set, 
* to detect linear (or "almost" linear) dependencies in your data, 
* to generate features for further processing of your data. 

In the following we present two options available in Heat to compute an **approximate truncated SVD for a large matrix**. 

### Randomized SVD 

The most easy option is so-called randomized SVD which allows you to specify the rank $r$ you are targeting as second positional argument. Here, we use randomized SVD with a certain number of oversamples and one power iteration in order to compute an approximation to the leading 10 singular values and vectors of the same matrix $A$ as above:   

In [ ]:
%%px 
Ur, Sr, Vr = ht.linalg.svd_randomized(X, 10, n_oversamples=10, power_iter=1)
Ur, Sr, Vr

### Hierarchical SVD 

Hierarchical SVD allows a bit more of freedom. Truncation takes place either w.r.t. a fixed truncation-rank (`heat.linalg.hsvd_rank`) or w.r.t. a desired accuracy (`heat.linalg.hsvd_rtol`). In the latter case it can be ensured that it holds for the "reconstruction error": 

$$
\frac{\lVert X - U U^T X \rVert_F}{\lVert X \rVert_F} \overset{!}{\leq} \text{rtol},
$$

where $U$ denotes the approximate left-singular vectors of $X$ computed by `heat.linalg.hsvd_rtol`. 

Let's first compute the truncated SVD by setting the relative tolerance: 

In [ ]:
%%px
# compute truncated SVD w.r.t. relative tolerance 
svd_with_reltol = ht.linalg.hsvd_rtol(X,rtol=1.0e-2,compute_sv=True,silent=False)
print("relative residual:", svd_with_reltol[3], "rank: ", svd_with_reltol[0].shape[1])

Alternatively, you can compute a truncated SVD with a fixed truncation rank:

In [ ]:
%%px
# compute truncated SVD w.r.t. a fixed truncation rank 
svd_with_rank = ht.linalg.hsvd_rank(X, maxrank=3,compute_sv=True,silent=False)
print("relative residual:", svd_with_rank[3], "rank: ", svd_with_rank[0].shape[1])

Once we have computed the truncated SVD, we can use it to approximate the original data matrix `X` by the truncated matrix `X_r`. 

Check out the plot below to see how Heat's truncated SVD algorithm scales with the number of MPI processes and size of the dataset.

<div>
  <img src=https://github.com/helmholtz-analytics/heat/blob/main/doc/source/_static/images/hSVD_bench_rank5.png?raw=true title="hSVD rank 5" width="30%" style="float:left"/>
  <img src=https://github.com/helmholtz-analytics/heat/blob/main/doc/source/_static/images/hSVD_bench_rank50.png?raw=true title="hSVD rank 50" width="30%" style="float:center "/>
  <img src=https://github.com/helmholtz-analytics/heat/blob/main/doc/source/_static/images/hSVD_bench_rank500.png?raw=true title="HSVD rank 500" width="30%" style="float:center"/>
</div>

## Exercises 

1. Try out different split combinations `split_A=0,1,None`, `split_B=0,1,None` for matrix-matrix multiplication. What do you observe for the split of the outcome? Does every combination take the same computing time? (Whats the likely cause for this?)

2. Compare the approximate singular values computed by randomized SVD / hierarchiacl SVD with the reference ones computed by full SVD. What do you observe for different choices of the parameters (number of oversamples, power iterations, truncation rank, truncation tolerance)? 